# RCOT on SUTs (c=i)

This notebook starts from an exemplary supply-use table (SUT) that features an equal number of commodities and industries (c=i). The cases of [c<i](./SU-RCOT_c=i.ipynb) and [c>i](./SU-RCOT_c+i-.ipynb) are illustrated in separate notebooks. All notebooks share the same basic structure. That is, an illustrative SUT is imported from a spreadsheet; solved for the cases of factor constraints being absent or present; and solved when new technologies are introduced.

<a id='toc'></a>

The outline of the current notebook is as follows:


0. [Data import and set-up](#data-import)
1. [Empirical system with factor constraints](#emp_f)\
    1.1 [As-is solution by inversion](#emp_f_inv)\
        1.1.1 [SUT-based](#emp_f_inv_sut)\
        1.1.2 [Construct_based](#emp_f_inv_ctc)\
    1.2 [As-is solution via linear program](#emp_f_lp)\
        1.2.1 [Absolute SU-RCOT](#emp_f_lp_abs)\
        1.2.2 [IO-RCOT](#emp_f_lp_rel)\
    1.3 [Method comparison for changed final demand](#comp)
3. [Adding technology alternatives to the SUT system](#add_jp)

<a id='data-import'></a>

## 0. Data import and set-up

In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using DataFrames

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for inversion-based or RCOT modelling based on SUT.structure or Constructs.construct
include("../src/Leontief.jl"); # Solves the model based on Leontief calculus
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c=i.xlsx")

XLSXFile("../data/SUT_c=i.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x4           A1:D4        
                  U+e 4x5           A1:E4        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D4"])
e = convert(Matrix{Float64}, xf["U+e!E2:E4"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"]

"mixed"

In [4]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: The SUT system is not given in monetary units such that no total industry output and dependent matrices may be calculated.
[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
# For completeness, let's check the total factor costs (in this unconstrained system)
sut.pii'*sut.f

1×1 Matrix{Float64}:
 261.90000000000003

Back to [table of contents](#toc)
<a id='emp_f'></a>

## 1. Empirical system with factor constraints

<a id='emp_f_inv'></a>

### 1.1 Solution by inversion
Let us first solve the system by inversion.

<a id='emp_f_inv_sut'></a>

#### 1.1.1 SUT-based

Because the given SUT is square, we can invert the SUT system directly to derive the industry activity levels and the total factor use. Since the data is given in mixed units, we can only use the absolute SUT data for the inversion-based model.

In [6]:
su_model_data = Model_data.SU(sut)
leo_sut_abs = Leontief.quantity(su_model_data, "abs");

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.
[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [7]:
leo_sut_abs.output

3×1 Matrix{Float64}:
 1.0
 0.9999999999999999
 1.0

In [8]:
leo_sut_abs.factor_use

4×1 Matrix{Float64}:
 34.0
 25.0
 26.0
 29.0

In [9]:
leo_sut_abs.factor_use_disagg_by_demand

4×3 Matrix{Float64}:
 8.60196  6.75172  18.6463
 3.45163  2.71584  18.8325
 8.53984  4.03338  13.4268
 5.40657  6.49906  17.0944

Let us briefly also look at the price model:

In [10]:
leo_sut_abs_p = Leontief.price(su_model_data, "abs");

[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [11]:
leo_sut_abs_p.com_price

3×1 Matrix{Float64}:
 2.1984456937063555
 2.1208619175194343
 2.387105494312442

In [12]:
leo_sut_abs_p.com_price_disagg_by_factors

3×4 Matrix{Float64}:
 0.397013  0.477918  0.886829  0.436685
 0.405103  0.488852  0.544506  0.682401
 0.329053  0.997016  0.533122  0.527914

<a id='emp_f_inv_ctc'></a>

#### 1.1.2 Construct-based
Since the data is given in mixed units, we may only apply certain constructs to derive a pseudo single production system. Let's go for the CTC, and we start with the transaction-based inversion:

(Note that if the data were given in a rectangular shape, no construct would be applicable - as described in our paper.)

In [13]:
ctc = Constructs.CTC(sut)
io_model_data = Model_data.IO(ctc)
leo_ctc_abs = Leontief.quantity(io_model_data, "abs");

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.
[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [14]:
leo_ctc_abs.output

3×1 Matrix{Float64}:
 0.9999999999999996
 0.9999999999999997
 0.9999999999999999

In [15]:
leo_ctc_abs.factor_use

4×1 Matrix{Float64}:
 33.999999999999986
 24.999999999999993
 25.999999999999986
 28.999999999999993

And now in coefficient form:

In [16]:
leo_ctc_rel = Leontief.quantity(io_model_data, "rel");

[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [17]:
leo_ctc_rel.output

3×1 Matrix{Float64}:
 112.99999999999997
  73.99999999999999
 126.99999999999999

In [18]:
leo_ctc_rel.factor_use

4×1 Matrix{Float64}:
 33.99999999999999
 24.999999999999996
 25.999999999999993
 28.999999999999996

Back to [table of contents](#toc)
<a id='emp_f_lp'></a>

### 1.2 Solution by LP
Let us now solve the system via a linear program.

#### 1.2.1 SU-RCOT
Since SU-RCOT solves in its relative form for the industry output, we cannot apply it to this case where the data is given in mixed units. Instead, we perform SU-RCOT in its absolute form:

In [19]:
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [20]:
su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.9
value.(var_con) = [1.0, 1.0, 1.0]
value.(demand_con) = [26.0, 20.0, 68.0]
value.(factor_con) = [34.0, 25.0, 26.0, 29.0]


Let us solve for completeness also the dual:

In [21]:
su_rcot("dual", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.9
value.(p) = [2.198445693706355, 2.1208619175194343, 2.3871054943124426]
value.(r) = [0.0, 0.0, 0.0, 0.0]
value.(profit_con) = [71.10000000000001, 81.60000000000001, 109.2]


#### 1.2.2 IO-RCOT

In [22]:
io_rcot_data = Model_data.IO(ctc);

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [23]:
# Model in absolute (transaction) form
io_rcot("primal", "abs", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.8999999999999
value.(var_con) = [0.9999999999999997, 0.9999999999999997, 0.9999999999999998]
value.(demand_con) = [26.0, 20.0, 68.0]
value.(factor_con) = [33.999999999999986, 24.999999999999993, 25.99999999999999, 28.99999999999999]


In [24]:
# Model in relative (coefficient) form
io_rcot("primal", "rel", io_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.9
value.(var_con) = [112.99999999999999, 73.99999999999999, 127.0]
value.(demand_con) = [26.0, 20.0, 68.0]
value.(factor_con) = [33.99999999999999, 25.0, 25.999999999999996, 29.0]


While we have shown now that all models yield the same results, we have done so only with the empirical base data.

Back to [table of contents](#toc)
<a id='comp'></a>

#### 1.3 Method comparison for changed final demand

Let us now alter the final demand, while keeping everything else equal. Importantly, this also means that the factor availabilities are now equal to the absolute factor use. We show only the primal models.

In [25]:
# Change final demand for commodity #c.2 and set up the data structures
sut.e[2] = 24
ctc = Constructs.CTC(sut)
io_model_data = Model_data.IO(ctc)
su_model_data = Model_data.SU(sut);

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.
[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [26]:
# Solution by inversion with absolute SUT data (relative not possible)
leo_sut_abs = Leontief.quantity(su_model_data, "abs")
@show leo_sut_abs.output
@show leo_sut_abs.factor_use;

leo_sut_abs.output = 

[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


[1.0053318107953797; 1.0660635074004126; 1.0248495761781369;;]
leo_sut_abs.factor_use = [35.35034476700777; 25.543168342922392; 26.806675743335234; 30.29981114673459;;]


In [27]:
# Solution by inversion with absolute construct data
leo_ctc_abs = Leontief.quantity(io_model_data, "abs")
@show leo_ctc_abs.output
@show leo_ctc_abs.factor_use;

leo_ctc_abs.output = [1.0207065420807746; 1.0660635074004126; 1.0306909207608217;;]
leo_ctc_abs.factor_use = [35.35034476700776; 25.54316834292239; 26.806675743335223; 30.299811146734587;;]


[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [28]:
# Solution by inversion with relative construct data
leo_ctc_rel = Leontief.quantity(io_model_data, "rel")
@show leo_ctc_rel.output
@show leo_ctc_rel.factor_use;

leo_ctc_rel.output = [115.33983925512756; 78.88869954763054; 130.89774693662434;;]
leo_ctc_rel.factor_use = [35.35034476700776; 25.543168342922392; 26.80667574333523; 30.29981114673459;;]


[ Info: You are about to perform an inversion-based imputation or impact analysis. Keep in mind that input variables are treated independently, thus potentially resulting in different results for a model in absolute or relative form.


In [29]:
# Solution by absolute IO-RCOT
io_rcot("primal", "abs", io_model_data);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 270.3834476700777
value.(var_con) = [1.020706542080775, 1.0660635074004126, 1.0306909207608217]
value.(demand_con) = [26.0, 24.0, 68.0]
value.(factor_con) = [35.35034476700777, 25.543168342922392, 26.806675743335234, 30.29981114673459]


In [30]:
# Solution by relative IO-RCOT
io_rcot("primal", "rel", io_model_data);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 270.3834476700777
value.(var_con) = [115.33983925512757, 78.88869954763054, 130.89774693662434]
value.(demand_con) = [26.0, 24.0, 68.0]
value.(factor_con) = [35.35034476700777, 25.543168342922396, 26.806675743335234, 30.29981114673459]


In [31]:
# Solution by absolute SU-RCOT
su_rcot("primal", "abs", su_model_data);

termination_status(model) = MathOptInterface.INFEASIBLE
primal_status(model) = MathOptInterface.NO_SOLUTION
dual_status(model) = MathOptInterface.INFEASIBILITY_CERTIFICATE
objective_value(model) = 270.38344767007777
value.(var_con) = [1.0053318107953797, 1.0660635074004128, 1.024849576178137]
value.(demand_con) = [26.0, 24.0, 68.0]
value.(factor_con) = [35.35034476700777, 25.5431683429224, 26.806675743335237, 30.2998111467346]


Unsurprisingly, the RCOT models are infeasible - because the physical factor limits would be transgressed if the increased final demand were to be satisfied. Since the inversion-based models are not constrained by factor availability, they yield a solution - although it may not be possible in the end...

Back to [table of contents](#toc)
<a id='add_jp'></a>

## 2. Adding technology alternatives under factor constraints
So far, we have only worked with the SUT system as imported and without considering factor constraints. Let us now add new technologies to it and see how substitution may or may not occur. We use the model in its absolute form.

In [32]:
# start from fresh data
su_rcot_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [33]:
# Define supply and use of the additional technology
V3_alt1 = [1 0 20]
U3_alt1 = [10 2 0]'
F3_alt1 = [1.5 1 0 1.1]';

In [34]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V3_alt1]
su_rcot_data.U = @views [su_rcot_data.U U3_alt1]
su_rcot_data.F = @views [su_rcot_data.F F3_alt1];

In [35]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 183.228515285881
value.(var_con) = [0.841260210035006, 0.997899649941657, 0.0, 5.445694282380398]
value.(demand_con) = [26.0, 24.0, 68.0]
value.(factor_con) = [33.545437572928826, 8.81073512252042, 23.595799299883318, 22.641278879813306]


We see that another set of technologies is activated.

Back to [table of contents](#toc)
<a id='add_jp_3mp_jp'></a>

---

We end here the exposition for the case (c=i).